In [495]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, SimpleRNN, LSTM

from keras.optimizers import Adam
from scipy import stats  # 과학용 계산 라이브러리

In [496]:
from sklearn.datasets import load_linnerud

linnerud = load_linnerud()
print(linnerud.DESCR)

.. _linnerrud_dataset:

Linnerrud dataset
-----------------

**Data Set Characteristics:**

    :Number of Instances: 20
    :Number of Attributes: 3
    :Missing Attribute Values: None

The Linnerud dataset is a multi-output regression dataset. It consists of three
exercise (data) and three physiological (target) variables collected from
twenty middle-aged men in a fitness club:

- *physiological* - CSV containing 20 observations on 3 physiological variables:
   Weight, Waist and Pulse.
- *exercise* - CSV containing 20 observations on 3 exercise variables:
   Chins, Situps and Jumps.

.. topic:: References

  * Tenenhaus, M. (1998). La regression PLS: theorie et pratique. Paris:
    Editions Technic.



In [497]:
linnerud

{'data': array([[  5., 162.,  60.],
        [  2., 110.,  60.],
        [ 12., 101., 101.],
        [ 12., 105.,  37.],
        [ 13., 155.,  58.],
        [  4., 101.,  42.],
        [  8., 101.,  38.],
        [  6., 125.,  40.],
        [ 15., 200.,  40.],
        [ 17., 251., 250.],
        [ 17., 120.,  38.],
        [ 13., 210., 115.],
        [ 14., 215., 105.],
        [  1.,  50.,  50.],
        [  6.,  70.,  31.],
        [ 12., 210., 120.],
        [  4.,  60.,  25.],
        [ 11., 230.,  80.],
        [ 15., 225.,  73.],
        [  2., 110.,  43.]]),
 'feature_names': ['Chins', 'Situps', 'Jumps'],
 'target': array([[191.,  36.,  50.],
        [189.,  37.,  52.],
        [193.,  38.,  58.],
        [162.,  35.,  62.],
        [189.,  35.,  46.],
        [182.,  36.,  56.],
        [211.,  38.,  56.],
        [167.,  34.,  60.],
        [176.,  31.,  74.],
        [154.,  33.,  56.],
        [169.,  34.,  50.],
        [166.,  33.,  52.],
        [154.,  34.,  64.],
        

In [498]:
df = pd.concat([pd.DataFrame(linnerud.data, columns=linnerud.feature_names),
                pd.DataFrame(linnerud.target, columns=linnerud.target_names)],
               axis=1)
df

,Chins,Situps,Jumps,Weight,Waist,Pulse
0,5.0,162.0,60.0,191.0,36.0,50.0
1,2.0,110.0,60.0,189.0,37.0,52.0
2,12.0,101.0,101.0,193.0,38.0,58.0
3,12.0,105.0,37.0,162.0,35.0,62.0
4,13.0,155.0,58.0,189.0,35.0,46.0
5,4.0,101.0,42.0,182.0,36.0,56.0
6,8.0,101.0,38.0,211.0,38.0,56.0
7,6.0,125.0,40.0,167.0,34.0,60.0
8,15.0,200.0,40.0,176.0,31.0,74.0
9,17.0,251.0,250.0,154.0,33.0,56.0


In [499]:
df.isna().sum()

Chins     0
Situps    0
Jumps     0
Weight    0
Waist     0
Pulse     0
dtype: int64

In [500]:
#Splitting the data into training and test test
x = df.drop(['Pulse'], axis=1)
y = df['Pulse']

from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=42) 

In [501]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((17, 5), (3, 5), (17,), (3,))

In [502]:
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std
x_test -= mean
x_test /= std

In [503]:
cnn_model = Sequential()
cnn_model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(5,1)))
cnn_model.add(Flatten())
cnn_model.add(Dense(4, activation='relu'))  # 은닉층 2
cnn_model.add(Dense(1, activation='linear'))
optimizer = Adam(learning_rate=0.05)

# 모델 컴파일
cnn_model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])


In [504]:
from keras.callbacks import ModelCheckpoint

k = 4
num_val_samples = len(x_train) // k

num_epochs = 100
all_mae_histories = []
checkpoint = ModelCheckpoint("best_kcnn.h5", save_best_only=True, verbose=0, monitor='val_mae')

for i in range(k):
    print(f"#{i+1}번째 폴드 처리중")
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)
    model = cnn_model
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=4, verbose=0, callbacks=checkpoint)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)

#1번째 폴드 처리중

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#2번째 폴드 처리중


/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#3번째 폴드 처리중
#4번째 폴드 처리중


In [505]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [506]:
model = keras.models.load_model("best_kcnn.h5")


test_mse_score, test_mae_score = model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 103ms/step - loss: 45.4529 - mae: 6.3285


In [507]:
# RNN 모델 생성
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, activation='relu', input_shape=(5, 1)))  # 은닉층 1
rnn_model.add(Dense(4, activation='relu'))  # 은닉층 2
rnn_model.add(Dense(1, activation='linear'))
optimizer = Adam(learning_rate=0.1)

# 모델 컴파일
rnn_model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])



In [508]:
from keras.callbacks import ModelCheckpoint

k = 4
num_val_samples = len(x_train) // k

num_epochs = 100
all_mae_histories = []
checkpoint = ModelCheckpoint("best_krnn.h5", save_best_only=True, verbose=0, monitor='val_mae')

for i in range(k):
    print(f"#{i+1}번째 폴드 처리중")
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)
    model = rnn_model
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=4, verbose=0, callbacks=checkpoint)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)

#1번째 폴드 처리중


/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#2번째 폴드 처리중


/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#3번째 폴드 처리중
#4번째 폴드 처리중


In [509]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [510]:
model = keras.models.load_model("best_krnn.h5")

test_mse_score, test_mae_score = model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 187ms/step - loss: 59.0680 - mae: 6.5216


In [511]:
# LSTM 모델 생성
lstm_model = Sequential()
lstm_model.add(LSTM(64, activation='relu', input_shape=(5, 1)))  # 은닉층 1
lstm_model.add(Dense(4, activation='relu'))  # 은닉층 2
lstm_model.add(Dense(1, activation='linear'))
optimizer = Adam(learning_rate=0.1)

# 모델 컴파일
lstm_model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

In [512]:
from keras.callbacks import ModelCheckpoint

k = 4
num_val_samples = len(x_train) // k

num_epochs = 100
all_mae_histories = []
checkpoint = ModelCheckpoint("best_klstm.h5", save_best_only=True, verbose=0, monitor='val_mae')

for i in range(k):
    print(f"#{i+1}번째 폴드 처리중")
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)
    model = rnn_model
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=4, verbose=0, callbacks=checkpoint)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)

#1번째 폴드 처리중


/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#2번째 폴드 처리중
#3번째 폴드 처리중


/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#4번째 폴드 처리중


/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [513]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [514]:
model = keras.models.load_model("best_klstm.h5")


test_mse_score, test_mae_score = model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 177ms/step - loss: 67.7941 - mae: 8.0143
